<h1 align=center style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
    کوویدکاوی
</font>
</h1>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3> می‌خواهیم داده‌های مربوط به وضعیت کووید-۱۹ در کشورهای مختلف جهان در یک روز از سال ۲۰۲۰ را کمی تحلیل کنیم و با پاسخ دادن به پرسش‌هایی از آن با کتابخانه‌ی <code>pandas</code> بیشتر کار کنیم.
</font>
</p>

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
مجموعه‌داده‌
</font>
</h2>



In [1]:
import pandas as pd

In [2]:
covid_data = pd.read_csv('covid19-11-10-2020.csv', index_col=0)
covid_data.shape

(3998, 14)

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
    <font face="vazir" size=3>
        <br>
       - ستون  <code>Country_Region</code>: مشخص است که نام کشور یا منطقه را مشخص می‌کند. دقت کنید که شاید لازم باشد بررسی کنید که آیا تعداد سطرها با تعداد کشورها برابر است و به بیان دیگر به ازای هر منطقه یا کشور فقط یک سطر دیتا وجود دارد یا خیر.
        <br>
        - ستون‌های <code>Long</code> و <code>Lat</code>: طول و عرض جفرافیایی کشور/منطقه را بیان می‌کند.
        <br>
        - ستون <code>Confirmed</code>: تعداد موارد تاییدشده‌ی ابتلا به ویروس
        <br>
        - ستون <code>Deaths</code>: تعداد موارد فوت‌شده
        <br>
        - ستون <code>Recovered</code>: تعداد موارد بهبودیافته
        <br>
        - ستون <code>Active</code>: تعداد موارد فعال (که از تفاضل مجموع فوتی‌ها و بهبودیافتگان از موارد تایید شده نیز به‌دست می‌آید.)
        <br>
        - ستون <code>Incident_Rate</code>: بیانگر تعداد موارد در هر ۱۰۰۰۰۰ نفر است.
</p>


<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
قسمت اول‌
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
     در ابتدا می‌خواهیم ببینیم ۱۰ کشور با بیشترین تعداد موارد  کدام‌اند. 
</font>
</p>

In [31]:
covid_data['Total_Cases'] = covid_data['Incident_Rate'].apply(lambda x: x*100000)

In [32]:
fr = covid_data.groupby('Country_Region').agg({'Total_Cases': 'sum'})

In [33]:
fr.reset_index(inplace=True)

In [34]:
fr = fr.sort_values(by="Total_Cases",ascending=False).head(10)

In [49]:
fr

,Country_Region,Total_Cases
185,US,1.060653e+12
146,Russia,1.084148e+10
24,Brazil,9.817550e+09
140,Peru,6.967909e+09
38,Colombia,6.661217e+09
166,Spain,5.766248e+09
36,Chile,4.348631e+09
86,Italy,3.359897e+09
187,Ukraine,3.171402e+09
80,India,3.116931e+09


In [48]:
fr.to_csv("top10cases.csv", header=False, index=True)

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
قسمت دوم
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
در ادامه می‌خواهیم ببینیم چه کشورهایی بیشترین نرخ ریکاوری را داشته‌اند. 
</font>
</p>

In [3]:
covid_data_sol = covid_data.groupby('Country_Region').agg({'Deaths': 'sum', 'Confirmed':'sum', 'Recovered':'sum'})

In [4]:
covid_data_sol.reset_index(inplace=True)

In [5]:
covid_data_sol2 = covid_data_sol[~((covid_data_sol['Confirmed'] == 0) & (covid_data_sol['Deaths'] == 0) & (covid_data_sol['Recovered'] == 0))]

In [6]:
covid_data_sol2

,Country_Region,Deaths,Confirmed,Recovered
0,Afghanistan,1574,42297,34954.0
1,Albania,579,25294,12353.0
2,Algeria,2077,63446,42626.0
3,Andorra,75,5477,4405.0
4,Angola,308,12816,6036.0
...,...,...,...,...
194,Vietnam,35,1226,1087.0
195,West Bank and Gaza,533,59422,51425.0
197,Yemen,601,2063,1375.0
198,Zambia,350,16997,16019.0


In [7]:
median_confrim = covid_data_sol2["Confirmed"].median()
median_confrim

23932.0

In [10]:
covid_data_sol2["Recovery_Rate"] = covid_data_sol2.apply(lambda row: recovery_rate_finder(row["Recovered"], row["Confirmed"]), axis=1)

C:\Users\Sabalan\AppData\Local\Temp\ipykernel_15804\4223501265.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covid_data_sol2["Recovery_Rate"] = covid_data_sol2.apply(lambda row: recovery_rate_finder(row["Recovered"], row["Confirmed"]), axis=1)


In [12]:
covid_data_sol2_1 = covid_data_sol2[covid_data_sol2["Confirmed"] > median_confrim]

In [11]:
covid_data_sol2

,Country_Region,Deaths,Confirmed,Recovered,Recovery_Rate
0,Afghanistan,1574,42297,34954.0,0.826394
1,Albania,579,25294,12353.0,0.488377
2,Algeria,2077,63446,42626.0,0.671847
3,Andorra,75,5477,4405.0,0.804272
4,Angola,308,12816,6036.0,0.470974
...,...,...,...,...,...
194,Vietnam,35,1226,1087.0,0.886623
195,West Bank and Gaza,533,59422,51425.0,0.865420
197,Yemen,601,2063,1375.0,0.666505
198,Zambia,350,16997,16019.0,0.942460


In [9]:
def recovery_rate_finder(Recovered, Confirmed):
    return Recovered / Confirmed

In [60]:
covid_data_sol2_1["Recovery_Rate"] = covid_data_sol2_1.apply(lambda row: recovery_rate_finder(row["Recovered"], row["Confirmed"]), axis=1)

C:\Users\Sabalan\AppData\Local\Temp\ipykernel_16936\4030070328.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covid_data_sol2_1["Recovery_Rate"] = covid_data_sol2_1.apply(lambda row: recovery_rate_finder(row["Recovered"], row["Confirmed"]), axis=1)


In [13]:
covid_data_sol2_1

,Country_Region,Deaths,Confirmed,Recovered,Recovery_Rate
0,Afghanistan,1574,42297,34954.0,0.826394
1,Albania,579,25294,12353.0,0.488377
2,Algeria,2077,63446,42626.0,0.671847
7,Argentina,34183,1262476,1081897.0,0.856964
8,Armenia,1609,108687,66835.0,0.614931
...,...,...,...,...,...
188,United Arab Emirates,518,144385,139701.0,0.967559
189,United Kingdom,68281,1237166,3010.0,0.002433
191,Uzbekistan,588,69027,66194.0,0.958958
193,Venezuela,834,95445,90359.0,0.946713


In [14]:
mean_recovery_rate= covid_data_sol2["Recovery_Rate"].mean()
mean_recovery_rate

0.7267428587109472

In [15]:
covid_data_sol2_1.sort_values(by="Recovery_Rate", ascending=False)

,Country_Region,Deaths,Confirmed,Recovered,Recovery_Rate
159,Singapore,28,58073,57985.0,0.998485
144,Qatar,233,134663,131733.0,0.978242
13,Bahrain,331,83811,81415.0,0.971412
68,Ghana,320,49302,47843.0,0.970407
188,United Arab Emirates,518,144385,139701.0,0.967559
...,...,...,...,...,...
17,Belgium,13561,507475,30504.0,0.060109
127,Netherlands,8211,426227,6185.0,0.014511
189,United Kingdom,68281,1237166,3010.0,0.002433
171,Sweden,6057,162240,0.0,0.000000


In [16]:
covid_data_sol2_2 = covid_data_sol2_1[covid_data_sol2_1["Recovery_Rate"]>mean_recovery_rate]

In [17]:
covid_data_sol2_2

,Country_Region,Deaths,Confirmed,Recovered,Recovery_Rate
0,Afghanistan,1574,42297,34954.0,0.826394
7,Argentina,34183,1262476,1081897.0,0.856964
9,Australia,907,27671,22298.0,0.805826
11,Azerbaijan,867,67392,50009.0,0.742061
13,Bahrain,331,83811,81415.0,0.971412
14,Bangladesh,6108,423620,341416.0,0.805949
16,Belarus,1016,108300,91646.0,0.846223
21,Bolivia,8808,142664,115199.0,0.807485
24,Brazil,162922,5726227,5183970.0,0.905303
28,Burma,1461,63241,47411.0,0.749688


In [18]:
covid_data_sol2_3 = covid_data_sol2_2[["Country_Region","Recovery_Rate"]].sort_values(by="Recovery_Rate", ascending=False)

In [128]:
covid_data_sol2_3.to_csv("recovery_rate.csv", header=False, index=True)

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
قسمت سوم
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
اکنون می‌خواهیم اوضاع سایر کشورها را با کشور خودمان مقایسه‌ای کنیم! ا
</font>
</p>

In [56]:
cor_ver = covid_data.copy()

In [57]:
cor_ver = cor_ver.groupby('Country_Region').agg({'Deaths': 'sum', 'Confirmed':'sum', 'Recovered':'sum'})

In [58]:
cor_ver

,Deaths,Confirmed,Recovered
Country_Region,,,
Afghanistan,1574,42297,34954.0
Albania,579,25294,12353.0
Algeria,2077,63446,42626.0
Andorra,75,5477,4405.0
Angola,308,12816,6036.0
...,...,...,...
West Bank and Gaza,533,59422,51425.0
Winter Olympics 2022,0,0,0.0
Yemen,601,2063,1375.0


In [59]:
cor_ver.reset_index(inplace=True)

In [60]:
cor_ver

,Country_Region,Deaths,Confirmed,Recovered
0,Afghanistan,1574,42297,34954.0
1,Albania,579,25294,12353.0
2,Algeria,2077,63446,42626.0
3,Andorra,75,5477,4405.0
4,Angola,308,12816,6036.0
...,...,...,...,...
195,West Bank and Gaza,533,59422,51425.0
196,Winter Olympics 2022,0,0,0.0
197,Yemen,601,2063,1375.0
198,Zambia,350,16997,16019.0


In [61]:
iran_confirmed = cor_ver[cor_ver["Country_Region"] == 'Iran']['Confirmed'].values[0]
iran_confirmed


703288

In [62]:
iran_death = cor_ver[cor_ver["Country_Region"] == 'Iran']['Deaths'].values[0]
iran_death

39202

In [63]:
final_cor_ver = cor_ver[(cor_ver['Confirmed'] > iran_confirmed) & (cor_ver['Deaths'] < iran_death)][['Country_Region','Confirmed','Deaths','Recovered']]

In [70]:
final_cor_ver.sort_values(by=['Confirmed'], ascending=False, inplace=True)

In [71]:
final_cor_ver

,Country_Region,Confirmed,Deaths,Recovered
146,Russia,1802762,30899,1341868.0
7,Argentina,1262476,34183,1081897.0
38,Colombia,1155356,33128,980866.0
164,South Africa,740254,19951,683194.0


In [129]:
final_cor_ver.to_csv("vsiran.csv", header=False, index=True)

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
قسمت چهارم
</font>
</h2>


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
در ادامه می‌خواهیم ادعای سخنگوی وزارت بهداشت کشوری را ارزیابی کنیم که ادعا می‌کند کشورش با توجه به آمار فوتی‌هایشان در این روز، جزو ۲۰ درصد برتر جهان است
<br>
</p>

In [74]:
per20 = covid_data.copy()

In [137]:
per20

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2020-11-11 05:25:30,33.93911,67.709953,42297,1574,34954.0,5769.0,Afghanistan,108.653487,3.721304
1,NaN,NaN,NaN,Albania,2020-11-11 05:25:30,41.15330,20.168300,25294,579,12353.0,12362.0,Albania,878.935298,2.289080
2,NaN,NaN,NaN,Algeria,2020-11-11 05:25:30,28.03390,1.659600,63446,2077,42626.0,18743.0,Algeria,144.685270,3.273650
3,NaN,NaN,NaN,Andorra,2020-11-11 05:25:30,42.50630,1.521800,5477,75,4405.0,997.0,Andorra,7088.591212,1.369363
4,NaN,NaN,NaN,Angola,2020-11-11 05:25:30,-11.20270,17.873900,12816,308,6036.0,6472.0,Angola,38.994388,2.403246
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3993,NaN,NaN,Unknown,Ukraine,2020-11-11 05:25:30,NaN,NaN,0,0,0.0,0.0,"Unknown, Ukraine",0.000000,0.000000
3994,NaN,NaN,NaN,Nauru,2020-11-11 05:25:30,-0.52280,166.931500,0,0,0.0,0.0,Nauru,0.000000,0.000000
3995,NaN,NaN,Niue,New Zealand,2020-11-11 05:25:30,-19.05440,-169.867200,0,0,0.0,0.0,"Niue, New Zealand",0.000000,0.000000
3996,NaN,NaN,NaN,Tuvalu,2020-11-11 05:25:30,-7.10950,177.649300,0,0,0.0,0.0,Tuvalu,0.000000,0.000000


In [138]:
per20_group = per20.groupby('Country_Region').agg({'Deaths': 'sum', 'Confirmed':'sum', 'Recovered':'sum'})
per20_group.reset_index(inplace=True)

In [139]:
per20_group = per20_group[~(per20_group['Confirmed'] == 0)]

In [140]:
per20_group

,Country_Region,Deaths,Confirmed,Recovered
0,Afghanistan,1574,42297,34954.0
1,Albania,579,25294,12353.0
2,Algeria,2077,63446,42626.0
3,Andorra,75,5477,4405.0
4,Angola,308,12816,6036.0
...,...,...,...,...
194,Vietnam,35,1226,1087.0
195,West Bank and Gaza,533,59422,51425.0
197,Yemen,601,2063,1375.0
198,Zambia,350,16997,16019.0


In [141]:
per20_group_fillter = per20_group.sort_values(by='Confirmed')[int(per20_group.shape[0]/5):]

In [136]:
per20_group_fillter = per20_group_fillter[['Country_Region','death_ratio']].sort_values(by="death_ratio", ascending=True)[:int(per20_group_fillter.shape[0]/5)]

In [134]:
per20_group_fillter

,Country_Region,death_ratio
159,Singapore,0.000482
144,Qatar,0.001730
167,Sri Lanka,0.002786
111,Maldives,0.003408
23,Botswana,0.003446
188,United Arab Emirates,0.003588


In [110]:
def death_ratio_finder(dth, con):
    return dth/con

In [114]:
per20_group_fillter['death_ratio']=per20_group_fillter.apply(lambda row: death_ratio_finder(row['Deaths'], row['Confirmed']), axis=1)

In [130]:
per20_group_fillter.to_csv("20percent.csv", header=False, index=True)